In [56]:
import pandas as pd
import numpy as np
import re

from collections import defaultdict

In [57]:
# Import items data
items_25 = pd.read_csv("../Final Project/items_2025.csv")

In [58]:
# Append product title to the end of breadcrumb as final breadcrumb
items_25['breadcrumb_title'] = items_25['product_breadcrumb'] + ',' + items_25['product_title']

In [59]:
# Create a function to split breadcrumbs into list
def breadcrumbs(product_breadcrumb, store_id, product_id):
    # Remove unwanted characters (e.g., [ ] " ')
    clean_breadcrumb = re.sub(r'[\]\[\"\']', '', product_breadcrumb)
    
    # Define possible splitters
    possible_splitters = [',', '/', '>']

    # Replace all splitters with a common delimiter (e.g., space) for uniform splitting
    for splitter in possible_splitters:
        clean_breadcrumb = clean_breadcrumb.replace(splitter, '|')

    # Split the breadcrumb into parts based on spaces
    split_breadcrumb = [i.strip() for i in clean_breadcrumb.split('|') if i.strip()]  # Strip spaces and remove empty parts

    return {product_id: split_breadcrumb}


# Dictionary to store breadcrumbs structured by store_id
breadcrumbs_dict = {}

def breadcrumb_run(df):
    for i in range(len(df)):
        store_id = df.loc[i, 'store_id']
        product_id = df.loc[i, 'product_id']
        product_breadcrumb = df.loc[i, 'breadcrumb_title']
        
        # Ensure store_id exists in dictionary
        if store_id not in breadcrumbs_dict:
            breadcrumbs_dict[store_id] = {}

        # Process and store breadcrumbs using the breadcrumbs function
        breadcrumbs_dict[store_id].update(breadcrumbs(product_breadcrumb, store_id, product_id))

In [60]:
# Apply breadcrumb run to items
breadcrumb_run(items_25)

In [61]:
# Define keyword categories
keyword_sets = {
    "Other": {"fabric", "cigarette", "cigar", "meal", "frozen", "shaver adapter", "beard oil", "food", "candle", "candles", "birthday card", "unperfumed", "pet", "hand sanitiser"},
    "Body Wash": {"body wash", "shower gel", "body soap"},
    "Shampoo/Conditioner": {"shampoo", "hair wash", "hair cleanser", "conditioner", "hair softener", "hair mask", "hydrating mask", "hair moisturizer", "hair moisturiser"},
    "Razors": {"razor", "shaver", "trimmer", "epilator"},
    "Shaving Cream": {"shaving cream", "shaving foam", "shaving gel", "shaving lotion", "shave cream", "razor cream", "shaving butter", "pre-shave cream", "lathering cream", "shaving soap"},
    "Perfume/Cologne": {"perfume", "cologne", "eau de toilette", "eau de parfum", "parfum", "aftershave", "body mist"}, 
    "Deodorant": {"deodorant", "antiperspirant", "deodorizing spray", "body spray"},
    "Supplements": {"Vitamin", "Omega-3", "supplement", "enzyme", "energy tablet"},
    "Oral Care": {"toothpaste", "toothbrush", "mouthwash", "dental floss", "water pick", "whitening strips", "interspace brush", "interdental brush", "toothpowder", "flosser"},
    "Skincare": {"SPF","sunscreen","lotion", "moisturiser", "moisturizer", "ointment", "balm", "body butter", "face cleanser", "facial cleanser", "face cleansing", "facial cleansing", "lip balm", "face cream", "facial cream"},
    "Vape": {"vape","vaporisor","vaporizer", "vape pod", "vape liquid", "vaping pod", "vaping liquid", "e-cig"},
    "Hair Dye": {"hair dye", "hair color", "hair colour", "hair bleach"},
    "Adult Intimacy": {"condom", "lubricant", "adult toy", "vibrator", "adult toy", "adult device", "intimate gel"},
    #"Kids Toys": {"toy", "game", "play"},
    "Cleaning Liquid": {"disinfectant", "bleach", "degreaser", "sanitizer", "sanitiser", "surface spray", "floor cleaner", "floor spray", "counter cleaner", "counter spray", "carpet cleaner", "carpet spray", "glass spray", "glass cleaner", "mirror cleaner", "mirror spray", "steel cleaner", "steel spray"}
}

# Dictionary to store classification results: {store_id: {product_id: "Category"}}
categorized_products = defaultdict(dict)

for store_id, products in breadcrumbs_dict.items():
    for product_id, breadcrumb_list in products.items():
        if not breadcrumb_list:  # Skip empty breadcrumbs
            continue
        
        # If the second breadcrumb is "Perfumes" in Store 1, replace it with an empty string
        # There is an issue with TESCO where they put some non-perfume items under the perfume category. This removes those categories, so we don't pick up non-Perfumes
        if store_id == 1 and len(breadcrumb_list) > 2 and breadcrumb_list[1].lower() == "perfumes" and breadcrumb_list[2].lower() == "aftershaves & gift sets":
            breadcrumb_list[1] = ""  # Replace the second-to-last breadcrumb with an empty string
            breadcrumb_list[2] = ""  # Replace the third-to-last breadcrumb with an empty string
        
        # Iterate from last breadcrumb backwards
        classified = False
        for breadcrumb in reversed(breadcrumb_list):
            breadcrumb_lower = breadcrumb.lower()
            
            # Check each keyword category
            for category, keywords in keyword_sets.items():
                if any(keyword in breadcrumb_lower for keyword in keywords):
                    categorized_products[store_id][product_id] = category
                    classified = True
                    break  # Stop searching once classified
            
            if classified:
                break  # Stop searching once classified
        
        # If no match was found, classify as "Other"
        if not classified:
            categorized_products[store_id][product_id] = "Other"


In [62]:
# View breadcrumb categorization for "eye test" validation
breadcrumbs = defaultdict(dict)

for store_id, products in breadcrumbs_dict.items():
    for product_id, breadcrumb_list in products.items():
        if categorized_products[store_id][product_id] == "Body Wash":  # Only store Non-Food items
            breadcrumbs[store_id][product_id] = breadcrumb_list

# Print results
for store_id, products in breadcrumbs.items():
    print(f"Store {store_id} (Body Wash Items):")
    for product_id, breadcrumb in products.items():
        print(f"  Product {product_id}: {' | '.join(breadcrumb)}")
    print()

Store 3 (Body Wash Items):
  Product 910001941616: Toiletries & Beauty | Bath | Shower & Soap | Shower Gel & Body Wash | Shower Gel & Body Wash | Dove Deeply Nourishing Body Wash
  Product 1000203618197: Toiletries & Beauty | Sun Care & Travel | Travel Size Toiletries | 3 for 2 Travel Size Toiletries | Lynx Africa Shower Gel
  Product 1000289208969: Toiletries & Beauty | Womens Toiletries | Period Products | Feminine Washes & Accessories | ASDA Intimate Body Wash Tickled Pink
  Product 1000064175181: Baby | Toddler & Kids | Baby Toiletries & Healthcare | Baby | Toddler & Kids Bath | Baby & Childrens Bath & Shower | ASDA Scrubbly Bubbly 3-in-1 Raspberry Fragrance Hair | Bath & Body Wash
  Product 1000383118729: Baby | Toddler & Kids | Baby Toiletries & Healthcare | Baby | Toddler & Kids Bath | Baby & Childrens Bath & Shower | Aveeno Baby Daily Care Hair and Body Wash
  Product 1000383118714: Baby | Toddler & Kids | Baby Toiletries & Healthcare | Baby | Toddler & Kids Bath | Baby & Child

In [63]:
# Convert the categorized dictionary to a DataFrame
items = []

for store_id, products in categorized_products.items():
    for product_id, category in products.items():
        breadcrumb_list = breadcrumbs_dict.get(store_id, {}).get(product_id, [])
        breadcrumb_clean = " | ".join(breadcrumb_list)  # Combine the breadcrumb list into a single string
        items.append({
            "store_id": store_id,
            "product_id": product_id,
            "breadcrumb_clean": breadcrumb_clean,
            "category": category
        })

# Create the pandas DataFrame
items_bc = pd.DataFrame(items)

In [64]:
items_bc.groupby('category')['product_id'].count()

category
Adult Intimacy             95
Body Wash                 789
Cleaning Liquid           306
Deodorant                 772
Hair Dye                  409
Oral Care                 986
Other                  109060
Perfume/Cologne           139
Razors                    566
Shampoo/Conditioner      1599
Shaving Cream              84
Skincare                 1264
Supplements              1031
Vape                      480
Name: product_id, dtype: int64

In [65]:
# Import item descriptions
items_desc = pd.read_csv('/Users/sambickel-barlow/Desktop/PP422/Final Project/all_desc_processed.csv')

/var/folders/qz/pj0lh7817m3c9ydwgfrtbtf00000gn/T/ipykernel_5113/1525727173.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  items_desc = pd.read_csv('/Users/sambickel-barlow/Desktop/PP422/Final Project/all_desc_processed.csv')


In [66]:
# Merge item descriptions with product categories based on breadcrerumbs
items_merge = items_desc.merge(items_bc, how='left', on=['store_id','product_id'])

In [67]:
# limit items to those that are categorized
items_limit = items_merge[(~items_merge['category'].isna()) & (items_merge['category'] != 'Other')]

In [69]:
items_limit.to_csv('/Users/sambickel-barlow/Desktop/PP422/Final Project/categorized_items3.csv')

In [68]:
# See n for each category
items_limit.groupby('category')['product_id'].count()

category
Adult Intimacy           67
Body Wash               596
Cleaning Liquid         195
Deodorant               521
Hair Dye                234
Oral Care               690
Perfume/Cologne          85
Razors                  406
Shampoo/Conditioner    1035
Shaving Cream            72
Skincare                914
Supplements             820
Vape                    335
Name: product_id, dtype: int64